# Workflow-execution magics

* **Difficulty level**: easy
* **Time need to lean**: 15 minutes or less
* **Key points**:
  * SoS workflows can be embedded in Jupyter notebook
  * Magic `%run` executes workflows defined in the current cell
  * Magic `%sosrun` executes workflows defined in the entire notebook
  * Magic `%runfile` executes workflows defined in specified file

SoS Notebook is an IDE for SoS workflow and allows the development and execution of workflows in a Jupyter environment.

## Scratch workflow steps

SoS workflow is extended from Python 3.6. You can execute any Python statement using the SoS kernel. That is to say, **you can use a SoS kernel just like a Python3 kernel**.

For example, the following cell uses SoS to execute a python statement, which is considered as a simple SoS step without header.

In [1]:
greeting = "Hello world"
print(f'This is our first greeting: {greeting}')

This is our first greeting: Hello world

In addition to regular Python statements, you can use SoS-specific syntax, functions, and statements in SoS cells.

For example, the following cell uses statement `output` to specify step output, and a `sh` function written in [script format](script_format.html).

In [2]:
output: 'result.txt'
sh: expand=True
   echo {greeting} > {_output}

The statements are executed in a global SoS namespace so variables defined in another cell (`greeting`) can be used here.

Technically speaking, we have justed executed a single SoS step in a global SoS namespace. Such steps are called **scratch steps** because they do not contain a header.

<div class="bs-callout bs-callout-warning" role="alert">
  <p>A workflow cell, namely an SoS cell with a header, cannot be executed directly. Running the cells will produce no output.</p>  
</div>

In contrast, a **formal SoS step** is defined as a step with a header. **Formal SoS steps and workflows have to be executed by SoS magics or commands**. As a matter of fact, nothing will happen if you execute the following cell in jupyter.

In [3]:
[hello]
print('This is our first hello world workflow')

## Magic `%run`

Magic `%run` executes workflows defined in the current cell. SoS starts an external `sos` process, execute the workflow and displays the output in the notebook. For example, the `hello` workflow could be executed as follows: 

In [4]:
%run
[hello-world]
print('This is our first hello world workflow')

This is our first hello world workflow


**The workflow is executed independently and does not share any variables in the SoS kernel**. For example, you cannot use variable `greeting` in the workflow,In 

In [5]:
%env --expect-error

%run

[greet-wrong]
print(f'This is our first greeting: {greeting}')

ERROR: [greet-wrong]: [0]: 


---------------------------------------------------------------------------


NameError                                 Traceback (most recent call last)


script_3050659269724098055 in <module>


----> print(f'This is our first greeting: {greeting}')


NameError: name 'greeting' is not defined


RuntimeError: Workflow exited with code 1

To pass variables to these workflows, you will have to define variables as parameters and pass them from command line.

In [6]:
%run --greeting '{greeting}' -v0

parameter: greeting = str

[greet]
print(f'This is our first greeting: {greeting}')

> %run --greeting 'Hello world' -v0

[#] 1 step processed (1 job completed)


Note that SoS expands `{ }` in the `%sos` magic so the actual magic executed was `%run --greeting `Hello world` -v0`

<div class="bs-callout bs-callout-info" role="alert">
    <p>The verbosity argument <code>-v</code> of magics <code>%run</code>, <code>%sosrun</code> and command <code>sos run</code></b> accepts values </p>
    <ul>
        <li><code>-v 0</code>: Display no system messages except errors</li>
        <li><code>-v 1</code>: Display errors and warnings, and a text-based progress bar</li>
        <li><code>-v 2 (default)</code>: Display errors, warnings, and informational messages</li>
        <li><code>-v 3</code>: Display additional debug messages</li>
        <li><code>-v 4</code>: Display very verbose trace messages for development purposes</li>
     </ul>
</div>

## Magic `%sosrun`

A SoS notebook can have multiple workflow sections defined in multiple code cells. These sections constitute the content of the **embedded SoS script** of the notebook. For example, the following steps, defined in three separate cells, are all part of the embedded SoS script of this notebook.

In [7]:
[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'DEG.csv'
figure_file = 'output.pdf'

In [8]:
[plot_1]
sh: expand=True
    xlsx2csv {excel_file} > {csv_file}

In [9]:
[plot_2]
R: expand=True
    data <- read.csv('{csv_file}')
    pdf('{figure_file}')
    plot(data$log2FoldChange, data$stat)
    dev.off()

<div class="bs-callout bs-callout-primary" role="alert">
    <p>An <b>embed SoS script</b> of a SoS notebook consists of SoS sections in all SoS cells of a notebook.</p>  
</div>

The easiest way to view the embedded script of a SoS notebook is to use the `%preview --workflow` magic as follows (The option `-n` lists the script in the notebook instead of the console panel). As you can see, the embedded script consists of steps from the entire notebook.

In [10]:
%preview -n --workflow

#!/usr/bin/env sos-runner
#fileformat=SOS1.0

[hello]
print('This is our first hello world workflow')

[hello-world]
print('This is our first hello world workflow')

[greet-wrong]
print(f'This is our first greeting: {greeting}')

[greet]
print(f'This is our first greeting: {greeting}')

[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'DEG.csv'
figure_file = 'output.pdf'

[plot_1]
sh: expand=True
    xlsx2csv {excel_file} > {csv_file}

[plot_2]
R: expand=True
    data <- read.csv('{csv_file}')
    pdf('{figure_file}')
    plot(data$log2FoldChange, data$stat)
    dev.off()



<div class="bs-callout bs-callout-primary" role="alert">
  <p> The <code>%sosrun</code> magic execute workflows defined in the embedded SoS script of a notebook.</p>  
</div>

The `%sosrun` magic can be used to execute any of the workflows defined in the notebook. For example, the following magic execute the workflow `plot` defined in the above section. Because multiple workflows are defined in this notebook (`hello_world`, and `plot`), a workflow name is required for this magic.

In [11]:
%sosrun plot

null device 


          1 


## Magic `%runfile`

<div class="bs-callout bs-callout-primary" role="alert">
  <p> The <code>%runfile</code> magic executes a SoS script from specified file with specified option. Both SoS scripts (usually with extension <code>.sos</code>) and SoS notebooks (with extension <code>.ipynb</code>) are supported.</p>  
</div>

The third magic to execute SoS workflows in SoS Notebook is to use the `%runfile` magic, which execute workflows from a specified external file. For example, instead of using magic `%sosrun`, you can execute the current notebook with magic 

In [12]:
%runfile workflow_magics.ipynb plot

null device 


          1 


## Command `sos`

The `%sosrun` magic calls an external command `sos` to execute workflows defined in the notebook. Although for the sake of convenience we will use magic `%run` to execute workflows throughout this documentation, please remember that **you can execute the notebook using command `sos` from command line**.

![running notebook from command line](../media/sos_cmd_cli.png)


Alternatively, you can also write the workflow in a text file (usually with extension `.sos`) and execute it with command `sos run`:

![running script from command line](../media/sos_cmd_script_cli.png)


## Running workflows in background

<div class="bs-callout bs-callout-info" role="alert">
    <p>You can execute a workflow using magics <code>%run</code>, <code>%sosrun</code>, and <code>%runfile</code> in the background by adding a <code>&</code> at the end of the magic. The workflow will be executed in a queue while you can continue to work the notebook.</p>
</div>

SoS Notebook usually starts a workflow and waits until the workflow is completed. If the workflow takes a long time to execute, you can send workflows to a queue in which workflows will be executed one by one while you continue to work on the notebook. A status table will be displayed for each queued workflows and log messages and results will continue to send back to SoS Notebook.

In [13]:
%run -v1 &

import time
for i in range(5):
    print(i)
    time.sleep(2)

0


1


2


3


4


## Further reading

* [Inclusion of scripts](script_format.html)
* [How to define and execute basic forward-type workflows](basic_workflow.html)
* [Command line interface](cli.html)